<a href="https://colab.research.google.com/github/brlima25/prophet/blob/main/btc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install yfinance

In [7]:
#importando bibliotecas
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '${:,.2f}'.format

In [32]:
hj = datetime.today().strftime('%Y-%m-%d')
data_ini = '2020-10-01'
df_btc = yf.download('BTC-USD', data_ini, hj)
df_btc.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-08,"$41,561.46","$42,228.94","$40,672.28","$41,733.94","$41,733.94",28066355845
2022-01-09,"$41,734.73","$42,663.95","$41,338.16","$41,911.60","$41,911.60",21294384372
2022-01-10,"$41,910.23","$42,199.48","$39,796.57","$41,821.26","$41,821.26",32104232331
2022-01-11,"$41,819.51","$43,001.16","$41,407.75","$42,735.86","$42,735.86",26327648900
2022-01-12,"$42,783.98","$43,834.36","$42,529.41","$43,834.36","$43,834.36",29160413184


In [33]:
df_btc.reset_index(inplace=True)

In [34]:
df_btc

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-10-01,"$10,795.25","$10,933.62","$10,472.36","$10,619.45","$10,619.45",40023134100
1,2020-10-02,"$10,619.82","$10,657.84","$10,416.69","$10,575.97","$10,575.97",48661453918
2,2020-10-03,"$10,575.10","$10,598.94","$10,511.13","$10,549.33","$10,549.33",44660271563
3,2020-10-04,"$10,550.44","$10,686.00","$10,534.39","$10,669.58","$10,669.58",71251776995
4,2020-10-05,"$10,676.53","$10,793.51","$10,634.60","$10,793.34","$10,793.34",47537578009
...,...,...,...,...,...,...,...
464,2022-01-08,"$41,561.46","$42,228.94","$40,672.28","$41,733.94","$41,733.94",28066355845
465,2022-01-09,"$41,734.73","$42,663.95","$41,338.16","$41,911.60","$41,911.60",21294384372
466,2022-01-10,"$41,910.23","$42,199.48","$39,796.57","$41,821.26","$41,821.26",32104232331
467,2022-01-11,"$41,819.51","$43,001.16","$41,407.75","$42,735.86","$42,735.86",26327648900


In [35]:
df = df_btc[["Date", "Adj Close"]]
df.rename(columns = {'Date': 'ds', 'Adj Close': 'y'}, inplace=True)

In [36]:
df

,ds,y
0,2020-10-01,"$10,619.45"
1,2020-10-02,"$10,575.97"
2,2020-10-03,"$10,549.33"
3,2020-10-04,"$10,669.58"
4,2020-10-05,"$10,793.34"
...,...,...
464,2022-01-08,"$41,733.94"
465,2022-01-09,"$41,911.60"
466,2022-01-10,"$41,821.26"
467,2022-01-11,"$42,735.86"


In [37]:
# Gratfico, preço de fechamento
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['ds'], y = df['y']))

In [38]:
model = Prophet(seasonality_mode='multiplicative')
model.fit(df)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [39]:
#criando df com dados no futuro
df_futuro = model.make_future_dataframe(periods=10)
df_futuro.tail (20)

,ds
459,2022-01-03
460,2022-01-04
461,2022-01-05
462,2022-01-06
463,2022-01-07
464,2022-01-08
465,2022-01-09
466,2022-01-10
467,2022-01-11
468,2022-01-12


In [40]:
#previsão 
previsao = model.predict(df_futuro)
previsao

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2020-10-01,"$5,311.95",$-406.94,"$11,601.15","$5,311.95","$5,311.95",$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$0.00,$0.00,$0.00,"$5,304.89"
1,2020-10-02,"$5,582.57",$-486.11,"$12,164.84","$5,582.57","$5,582.57",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,"$5,587.74"
2,2020-10-03,"$5,853.19",$-344.37,"$11,833.80","$5,853.19","$5,853.19",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,"$5,865.15"
3,2020-10-04,"$6,123.81",$273.05,"$12,510.78","$6,123.81","$6,123.81",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,"$6,136.58"
4,2020-10-05,"$6,394.43",$287.00,"$12,360.19","$6,394.43","$6,394.43",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,"$6,397.40"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,2022-01-18,"$49,939.20","$42,985.47","$55,641.61","$49,899.15","$49,998.28",$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$0.00,$0.00,$0.00,"$49,753.32"
475,2022-01-19,"$49,884.40","$43,909.65","$56,307.22","$49,801.74","$49,990.35",$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$0.00,$0.00,$0.00,"$49,861.11"
476,2022-01-20,"$49,829.60","$43,463.33","$56,057.44","$49,699.21","$49,965.30",$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$-0.00,$0.00,$0.00,$0.00,"$49,763.34"
477,2022-01-21,"$49,774.80","$43,801.63","$56,027.18","$49,588.17","$49,958.66",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,"$49,820.84"


In [41]:
previsao[['ds','yhat','yhat_lower', 'yhat_upper']].tail(100)

,ds,yhat,yhat_lower,yhat_upper
379,2021-10-15,"$55,196.31","$49,204.03","$61,364.28"
380,2021-10-16,"$55,203.06","$49,335.48","$61,700.22"
381,2021-10-17,"$55,150.48","$48,820.88","$61,533.58"
382,2021-10-18,"$55,006.49","$49,080.68","$61,438.69"
383,2021-10-19,"$54,721.65","$48,663.57","$60,810.74"
...,...,...,...,...
474,2022-01-18,"$49,753.32","$42,985.47","$55,641.61"
475,2022-01-19,"$49,861.11","$43,909.65","$56,307.22"
476,2022-01-20,"$49,763.34","$43,463.33","$56,057.44"
477,2022-01-21,"$49,820.84","$43,801.63","$56,027.18"


In [42]:
#grafico
plot_plotly(model, previsao)

In [43]:
  plot_components_plotly(model,previsao)